In [43]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
# 구글 드라이브에 git clone
%cd /content/drive/MyDrive/pythorch
!git clone https://github.com/deeplearningzerotoall/PyTorch.git

/content/drive/MyDrive/pythorch
fatal: destination path 'PyTorch' already exists and is not an empty directory.


$$ H(x) = Wx + b $$

$$ cost(W, b) = \frac{1}{m} \sum^m_{i=1} \left( H(x^{(i)}) - y^{(i)} \right)^2 $$

### import

In [45]:
import torch
# nn : neural network
import torch.nn.functional as F
import torch.optim as optim

In [46]:
# 파이토치에도 난수 생성기가 있다.

torch.manual_seed(1)    # 1로 고정(실행할때마다 동일한 결과를 보기 위함)

### data

In [47]:
# y==x
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

기본적으로 PyTorch는 NCHW 형태이다.

In [48]:
# NCHW
# NUMBER, CHANNEL, HEIGHT, WIDTH
# 텐서플로우는 HEIGHT,WIDTH,CHANNEL임

# (100,3,480,720) 100장 3채널 480*720  -> 파이토치
# (480,720,3) ->

## Weight Initialization

In [49]:
W = torch.zeros(1, requires_grad=True)
print(W)
# torch  type으로 1개 파라미터를 0으로 초기화
# 학습시에 업데이트하는 가중치

tensor([0.], requires_grad=True)


In [50]:
# requires_grad = True
# 값을 계속 업데이터 할것인지 ?

In [51]:
b = torch.zeros(1, requires_grad=True)
print(b)

tensor([0.], requires_grad=True)


In [52]:
hypothesis = x_train * W + b
print(hypothesis)

# w와 b를 모두 0으로 초기화했으니 가설도 0

tensor([[0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)


In [53]:
print(hypothesis - y_train)

tensor([[-1.],
        [-2.],
        [-3.]], grad_fn=<SubBackward0>)


In [54]:
print((hypothesis - y_train) ** 2)

tensor([[1.],
        [4.],
        [9.]], grad_fn=<PowBackward0>)


In [55]:
cost = torch.mean((hypothesis - y_train) ** 2)
print(cost)

tensor(4.6667, grad_fn=<MeanBackward0>)


## Gradient Descent

In [56]:
# 경사하강법 : sgd, 학습하는 파라미터 w,b 학습륧 lr
optimizer = optim.SGD([W, b], lr=0.01) # lr -> learing rate
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [57]:
optimizer.zero_grad()   # 기울기를 계산하기전에 0으로 초기화
print(optimizer)
cost.backward() # 오차역전파법으로 w,b값을 업데이트
print(cost)
optimizer.step()    # 한걸음 이동
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)
tensor(4.6667, grad_fn=<MeanBackward0>)
SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [59]:
print(W)
print(b)

tensor([0.0933], requires_grad=True)
tensor([0.0400], requires_grad=True)


In [68]:
# 데이터
# x * 2 + 1
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[3], [5], [7]])
# y_train = torch.FloatTensor([[1], [2], [3]])

# 모델 초기화(0으로)
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정(초기화?)
optimizer = optim.SGD([W, b], lr=0.01)
# optimizer = optim.SGD([W, b], lr=0.005) # 한번에 갈수 있는 보폭이 작아지니깐 에폭을 키워줘야 함
# optimizer = optim.SGD([W, b], lr=1)   # 너무 커지면 학습이 되질 않아

# 1000번돌려
nb_epochs = 3000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = x_train * W + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    # 이전 학습과정에 계산된 그래디언트를 누적
    # 새로운 학습과정이 시작되면 그것을 초기화 해줘야함  -> 그래디언트 누적 방지
    cost.backward()
    # 손실함수에 대한 모델 가중치의 그래디언트를 계산하는데 사용
    # 각 가중치에 대한 손실함수의 변화량을 계산
    # 계산된 그래디언트는 각 가중치의 grad 속성에 저장됨
    optimizer.step()
    # 가중치를 업데이트
    # 위에서 계산된 그래디언트를 사용하여 가중치를 조정
    # 일반적으로 가중치는 학습률에 따라 그래디언트 방향으로 조정되어 손실을 줄이는 방향으로 모델이 개선됨


    # 정리!
    # 이전의 그래디언트를 지우고(zero_grad())
    # 역전파를 통해 새로운 그래디언트를 계산(backward())
    # 옵티마이저를 통해 가중치를 업데이트(step())

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()
        ))

Epoch    0/3000 W: 22.667, b: 10.000 Cost: 27.666666
Epoch  100/3000 W: nan, b: nan Cost: nan
Epoch  200/3000 W: nan, b: nan Cost: nan
Epoch  300/3000 W: nan, b: nan Cost: nan
Epoch  400/3000 W: nan, b: nan Cost: nan
Epoch  500/3000 W: nan, b: nan Cost: nan
Epoch  600/3000 W: nan, b: nan Cost: nan
Epoch  700/3000 W: nan, b: nan Cost: nan
Epoch  800/3000 W: nan, b: nan Cost: nan
Epoch  900/3000 W: nan, b: nan Cost: nan
Epoch 1000/3000 W: nan, b: nan Cost: nan
Epoch 1100/3000 W: nan, b: nan Cost: nan
Epoch 1200/3000 W: nan, b: nan Cost: nan
Epoch 1300/3000 W: nan, b: nan Cost: nan
Epoch 1400/3000 W: nan, b: nan Cost: nan
Epoch 1500/3000 W: nan, b: nan Cost: nan
Epoch 1600/3000 W: nan, b: nan Cost: nan
Epoch 1700/3000 W: nan, b: nan Cost: nan
Epoch 1800/3000 W: nan, b: nan Cost: nan
Epoch 1900/3000 W: nan, b: nan Cost: nan
Epoch 2000/3000 W: nan, b: nan Cost: nan
Epoch 2100/3000 W: nan, b: nan Cost: nan
Epoch 2200/3000 W: nan, b: nan Cost: nan
Epoch 2300/3000 W: nan, b: nan Cost: nan
Epoc